## Постановка задачи

Написать программу вычисления матричного выражения:

$\mathbf{A} = \mathbf{B}\ \mathbf{C}^2 + M(\mathbf{C})\ \mathbf{I} + \mathbf{I} + D(\mathbf{B})\ \mathbf{E}$,

где $\mathbf{B}$, $\mathbf{C}$ – квадратные плотные матрицы, элементы которых имеют тип double, причем элементы матрицы $\mathbf{C}$ задаются с помощью генератора псевдослучайных чисел, $\mathbf{I}$ – единичная матрица, $\mathbf{E}$ – полностью заполненная матрица, все элементы которой равны единице, $M(\mathbf{C)}$ – среднее значений элементов матрицы $\mathbf{C}$, $D(\mathbf{C)}$ – дисперсия элементов матрицы $\mathbf{C}$. Распараллелить эту программу с помощью OpenMP (parallel, task). Исследовать зависимость масштабируемости параллельной версии программы от ее вычислительной трудоемкости (размера матриц).  
Проверить корректность параллельной версии.  
Проверка закона Амдала. Построить зависимость ускорение:число потоков для заданного примера.

## Программно-аппаратная конфигурация тестового стенда

Информация о программно-аппаратной конфигурации тестового стенда была получена c помощью следующего shell-скрипта:

```sh
echo -n "OS: "; cat /etc/os-release | grep PRETTY_NAME \
     | cut -d= -f2 | tr -d '"'
echo -n "GCC Version: "; gcc --version | head -n 1
echo -n "CPU Model: "; lscpu | grep "Model name" \
     | awk -F: '{print $2}' | xargs
echo -n "Logical Cores: "; lscpu | grep "^CPU(s):" \
     | awk -F: '{print $2}' | xargs
echo -n "RAM: "; free -h --si | awk '/Mem:/ {print $2}'
```

Результат работы скрипта получения программно-аппаратной конфигурации:

In [ ]:
!echo -n "OS: "; cat /etc/os-release | grep PRETTY_NAME \
     | cut -d= -f2 | tr -d '"'
!echo -n "GCC Version: "; gcc --version | head -n 1
!echo -n "CPU Model: "; lscpu | grep "Model name" \
     | awk -F: '{print $2}' | xargs
!echo -n "Logical Cores: "; lscpu | grep "^CPU(s):" \
     | awk -F: '{print $2}' | xargs
!echo -n "RAM: "; free -h --si | awk '/Mem:/ {print $2}'

## Метод

Алгоритм решения:

- вычисление $M(\mathbf{C})$;
- вычисление $D(\mathbf{B})$;
- вычисление $\mathbf{C}^2$;
- вычисление $\mathbf{B}\ \mathbf{C}^2$;
- прибавление $1 + M(\mathbf{C})$ к каждому элементу главной диагонали $\mathbf{B}\ \mathbf{C}^2$;
- прибавление $D(\mathbf{B})$ к каждому элементу полученной на предыдущем шаге матрицы.

Для вычисления произведения матриц используется наивный алгоритм, основанный на определении произведения матриц $(\mathbf{A}\ \mathbf{B})_{i,j} = \sum_{k=1}^n \mathbf{A}_{i,k}\ \mathbf{B}^T_{j,k}$. Правый множитель транспонируется перед вычислением произведения, чтобы эффективнее использовать кэши процессора.

## Реализация

Решение реализовано на языке программирования C и находится в приложении к данному отчёту. Также в приложении к отчёту находится make-файл, используемый для сборки двух исполняемых файлов:

- `main` – решает поставленную задачу;
- `save_random_matrix` – генерирует случайные квадратные матрицы указанного размера.

При вычислении каждой матричной операции внешний цикл распараллеливается с помощью директивы OpenMP `parallel for`.

## Эксперимент

В данном разделе приведены результаты экспериментов, призванных ответить на три исследовательских вопроса:

1. Корректно ли разработанная реализация работает в параллельном режиме?
2. Как разработанная реализация масштабируется при увеличении размера матриц?
3. Выполняется ли для разработанной реализации закон Амдала?

### Корректность

Сначала была проверена корректность работы последовательной версии программы на следующих небольших матрицах $\mathbf{B}$ и $\mathbf{C}$ размера $2 \times 2$, правильный ответ для которых был посчитан вручную:

$$
\mathbf{B} = \begin{bmatrix} 1 & 2 \\ 3 & 4 \end{bmatrix}, \quad
\mathbf{C} = \begin{bmatrix} 5 & 6 \\ 7 & 8 \end{bmatrix},
$$
$$
M(\mathbf{C}) = 6.5, \quad D(\mathbf{B}) = 1.25,
$$
$$
\mathbf{A} = \mathbf{B} \cdot \mathbf{C}^2 + M(\mathbf{C}) \cdot \mathbf{I} + \mathbf{I} + D(\mathbf{B}) \cdot \mathbf{E},
$$
$$
\mathbf{A} = \begin{bmatrix} 257.75 & 291.25 \\ 566.25 & 666.75 \end{bmatrix}.
$$


In [ ]:
%pip install matplotlib
%pip install nbconvert
%pip install pandas
%pip install scipy
%pip install tabulate

In [ ]:
import subprocess
import re
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import filecmp

from getpass import getpass
from IPython.display import Markdown, display
from scipy.optimize import curve_fit

In [ ]:
subprocess.run(['make'], check=True)

Для проверки соответствия результатов работы последовательной версии программы результатам, полученным вручную, использовался следующий Python-скрипт:

```python
subprocess.run(
    [
        './main',
        'data/B_sample.txt',
        'data/C_sample.txt',
        'data/A_sample_actual.txt',
        '1' ## number of threads
    ],
    stdout=subprocess.PIPE, text=True, check=True
)
actual_path = "data/A_sample_actual.txt"
expected_path = "data/A_sample_expected.txt"

print(f"'{actual_path}' file content:")
!cat {actual_path} 

if (filecmp.cmp(actual_path, expected_path)):
    print("Seqential version results match manual results")
else:
    print("Seqential and manual results diverge!")
```

Далее приведён результат работы скрипта проверки корректности последовательной версии программы на рассмотренном примере:

In [ ]:
subprocess.run(
    [
        './main',
        'data/B_sample.txt',
        'data/C_sample.txt',
        'data/A_sample_actual.txt',
    ],
    stdout=subprocess.PIPE, text=True, check=True
)
actual_path = "data/A_sample_actual.txt"
expected_path = "data/A_sample_expected.txt"

print(f"'{actual_path}' file content:")
!cat {actual_path} 

if (filecmp.cmp(actual_path, expected_path)):
    print("Seqential version results match manual results")
else:
    print("Seqential and manual results diverged!")

После этого был запущен следующий Python-скрипт, генерирующий псевдослучайные матрицы размера $3000 \times 3000$ и сверяющий для этих матриц результаты работы последовательной и параллельной версий программы (реализацию функции `run_experiment` можно найти в приложении).

```python
_, _, seq_output = run_experiment(matrix_size=3000, num_threads=1)
_, _, par_output = run_experiment(matrix_size=3000, num_threads=64)
if (filecmp.cmp(seq_output, par_output)):
    print("Parallel and sequential results match")
else:
    print("Parallel and sequential results diverge!")
```

Далее приведён результат работы данного скрипта:

In [ ]:
def run_experiment(
        matrix_size,
        num_procs,
        seed=42,
        min_value=-1.0,
        max_value=1.0,
        reruns = 5,
        data_dir = 'data'
):
    """
    Returns the following triple:
      (mean_time, time_std, path_to_output_file)
    """
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    B_filename = os.path.join(data_dir, f'B_{matrix_size}.txt')
    C_filename = os.path.join(data_dir, f'C_{matrix_size}.txt')
    A_filename = os.path \
        .join(data_dir, f'A_{matrix_size}_{num_procs}.txt')
    
    subprocess.run(['./save_random_matrix', B_filename, \
                    str(matrix_size), str(min_value), \
                    str(max_value), str(seed)], check=True)
    
    subprocess.run(['./save_random_matrix', C_filename, \
                    str(matrix_size), str(min_value), \
                    str(max_value), str(seed+1)], check=True)
    
    times = []

    for _ in range(reruns):
        result = subprocess.run(['mpirun', '-n', str(num_procs), 'main', \
                                 B_filename, C_filename,A_filename], \
                                stdout=subprocess.PIPE, text=True)
        match = re.search(r'Computation time: ([\d\.]+) seconds', \
                          result.stdout)
        if match:
            times.append(float(match.group(1)))
        else:
            raise RuntimeError("Failed to extract computation time")
        
    return np.mean(times), np.std(times, ddof=1), A_filename

In [ ]:
_, _, seq_output = run_experiment(matrix_size=3000, num_procs=1)
_, _, par_output = run_experiment(matrix_size=3000, num_procs=20)
if (filecmp.cmp(seq_output, par_output)):
    print("Parallel and sequential results match")
else:
    print("Parallel and sequential results diverge!")

### Масштабируемость

Здесь и далее время работы указывается без учёта времени генерации входных данных и времени, затраченного на операции ввода-вывода.

Для оценки масштабируемости программы от её вычислительной трудоемкости (размера матриц) был выполнен следующий Python-скрипт:
```python
sizes = list(range(2000, 7000, 600))
num_threads = 12

times_for_sizes = []
for size in sizes:
    time, _ = run_experiment(size, num_threads)
    times_for_sizes.append(time)
    print(f"N: {size}, time: {time:.6f} seconds")

pd.DataFrame({
    'N': sizes,
    'Time (seconds)': times_for_sizes
})
```

В результате работы скрипта была построена следующая таблица, показывающая как время работы параллельной версии программы для матриц размера $N \times N$ увеличивается с ростом $N$ при использовании 12 потоков (после <<$\pm$>> указывается выборочное стандартное отклонение по результатам пяти замеров):

In [ ]:
sizes = list(range(2000, 7000, 600))
num_threads = 12

mean_times_for_sizes = []
stds_for_sizes = []
for size in sizes:
    mean_time, time_std, _ = run_experiment(size, num_threads)
    mean_times_for_sizes.append(mean_time)
    stds_for_sizes.append(time_std)
    print(f"N: {size}, time: {mean_time:.6f} +- {time_std:.6f} seconds")

In [ ]:
mean_time_plus_minus_std = [
    f"{mean:.2f} ± {std:.2f}" 
    for mean, std in zip(mean_times_for_sizes, stds_for_sizes)
]

display(Markdown(pd.DataFrame({
    'N': sizes,
    'Time ± std (seconds)': mean_time_plus_minus_std
}).round(2).to_markdown(index=False)))

Также по данной таблице был построен следующий график, более наглядно показывающий связь размера матрицы $N$ и времени работы:

In [ ]:
plt.figure(figsize=(10,6))

plt.errorbar(sizes, mean_times_for_sizes, yerr=stds_for_sizes, fmt='o-', capsize=5)
plt.title('Computation Time for N-by-N Matrices')
plt.xlabel('N')
plt.ylabel('Computation Time ± std (seconds)')
plt.grid(True)

plt.show()

С помощью следующего Python-скрипта был определён коэффициент наклона прямой, отражающей зависимость времени работы программы от размера задачи в логарифмическом масштабе.

```python
slope, _ = np.polyfit(np.log(sizes), np.log(mean_times_for_sizes), deg=1)
print(f"Slope on a log-log plot: {slope:.2f}")
```

В результате выполнения скрипта была получена следующая оценка угла наклона, соответствующая степени экспоненты в зависимости времени выполнения от размера задачи:

In [ ]:
slope, _ = np.polyfit(np.log(sizes), np.log(mean_times_for_sizes), deg=1)
print(f"Slope on a log-log plot: {slope:.2f}")

### Проверка закона Амдала

Для оценки зависимости времени работы от числа потоков и проверки закона Амдала был выполнен следующий Python-скрипт:

```python
matrix_size = 3000
max_threads = 24

threads_list = list(range(1, max_threads+1))
times_for_threads = []

for num_threads in threads_list:
    time, _, _ = run_experiment(matrix_size, num_threads)
    times_for_threads.append(time)

pd.DataFrame({
    'Number of Threads': threads_list,
    'Measured Speedup': speedups,
    f"Amdahl's law (P={parallel_part:.4f})": predicted_speedups
}).round(2)
```

В следующее таблицы приведены средние ускорения по результатам пяти замеров для матриц размера $3000 \times 3000$. Также в таблице для каждого рассмотренного числа потоков указано ускорение, предсказываемое законом Амдала для значения параметра $P$, минимизирующего среднеквадратичную ошибку законом Амдала для тех случаев, когда потоков меньше числа логических потоков.

In [ ]:
matrix_size = 3000
max_threads = 24

threads_list = list(range(1, max_threads+1))
times_for_threads = []

for num_threads in threads_list:
    time, _, _ = run_experiment(matrix_size, num_threads)
    times_for_threads.append(time)
    print(f'Number of threads: {num_threads}, time: {time:.6f} seconds')

In [ ]:
base_time = times_for_threads[0]
speedups = [base_time / t for t in times_for_threads]

def amdahl_law(N, P):
    return 1 / ((1 - P) + P / N)

params, _ = curve_fit(amdahl_law, threads_list[:11], speedups[:11], bounds=(0, 1))
parallel_part = params[0]

predicted_speedups = [amdahl_law(N, parallel_part) for N in threads_list]

display(Markdown(pd.DataFrame({
    'Number of Threads': threads_list,
    'Measured Speedup': speedups,
    f"Amdahl's law (P={parallel_part:.4f})": predicted_speedups
}).round(2).to_markdown(index=False)))

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(threads_list, speedups, marker='o', label='Measured Speedup')
plt.plot(threads_list, predicted_speedups, marker='x', linestyle='--', label=f"Amdahl's law (P={parallel_part:.4f})")
plt.title('Speedup vs Number of Threads')
plt.xlabel('Number of Threads')
plt.ylabel('Speedup')
plt.legend()
plt.grid(True)
plt.show()

## Выводы

Эмпирическим путём установлено, что:

- параллельная реализация работает корректно на рассмотренном примере;
- время работы параллельной реализации растёт примерно пропорционально $N^{3.03}$;
- закон Амдала выполняется для $P=0.9910$, когда число потоков меньше 12.

Объяснение расхождений с теорией:

- расхождение с теоретической оценкой сложности $\mathcal{O}(N^3)$ объясняется конечностью размера кэшей-процессора: с ростом размера матрицы всё меньшая часть матрицы помещается в кэши, что приводит к замедлению реализации;
- невыполнение закона Амдала при использовании более чем 12 потоков объяснятся тем, что используемый процессор имеет 12 логических потоков;
- небольшое отклонение от закона Амдала при 12 потоках объясняется тем, что часть процессорного времени тратится на выполнение фоновых задач.

## Приложение

Листинг 1: Файл `src/main.c` --- реализация алгоритма вычисления $\mathbf{A} = \mathbf{B}\ \mathbf{C}^2 + M(\mathbf{C})\ \mathbf{I} + \mathbf{I} + D(\mathbf{B})\ \mathbf{E}$.
```c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>
#include <string.h>
#include "matrix_utils.h"

void print_help() {
    printf("Computes and saves "
        "A = B * C^2 + M(C) * I + I + D(B) * E\n"
        "Usage: ./main <B_matrix_file> <C_matrix_file> "
        "<output_A_matrix_file> <num_threads>\n"
        "Matrix file for N-by-N matrix should be in the "
        "following format:\n"
        "First line: N\n"
        "Next N lines: each line contains N space-separated "
        "double values\n");
}

int main(int argc, char *argv[]) {
    if (argc != 5) {
        print_help();
        return EXIT_FAILURE;
    }

    const char *B_filename = argv[1];
    const char *C_filename = argv[2];
    const char *A_filename = argv[3];
    int num_threads = atoi(argv[4]);

    Matrix *B = read_matrix(B_filename);
    Matrix *C = read_matrix(C_filename);
    int n = B->n;

    if (C->n != n) {
        fprintf(stderr, "Matrix sizes do not match.\n");
        exit(EXIT_FAILURE);
    }

    omp_set_num_threads(num_threads);

    double start_time = omp_get_wtime();

    double M_C = compute_mean(C);
    double D_B = compute_variance(B);

    Matrix *C_squared = multiply_matrices(C, C);
    Matrix *result = multiply_matrices(B, C_squared);

    add_scalar_to_diagonal(result, M_C + 1.0);
    add_scalar_to_all_elements(result, D_B);

    double end_time = omp_get_wtime();
    double computation_time = end_time - start_time;
    printf("Computation time: %f seconds\n", computation_time);

    write_matrix(A_filename, result);

    free_matrix(B);
    free_matrix(C);
    free_matrix(C_squared);
    free_matrix(result);

    return EXIT_SUCCESS;
}
```

Листинг 2: Файл `src/save_random_matrix.c` --- реализация генерации случайных квадратные матриц.
```c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include "matrix_utils.h"

void print_help() {
    printf("Saves random N-by-N matrix\n"
           "Usage: ./save_random_matrix <output_matrix_file> <N> "
           "<min_value> <max_value> <seed>\n");
}

int main(int argc, char *argv[]) {
    if (argc != 6) {
        print_help();
        return EXIT_FAILURE;
    }

    const char *output_filename = argv[1];
    int n = atoi(argv[2]);
    double min_value = atof(argv[3]);
    double max_value = atof(argv[4]);
    int seed = atoi(argv[5]);

    srand(seed);

    Matrix *matrix = allocate_matrix(n);

    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < n; ++j) {
            double r = ((double)rand() / RAND_MAX);
            matrix->data[i][j] = min_value
                + r * (max_value - min_value);
        }
    }

    write_matrix(output_filename, matrix);

    free_matrix(matrix);

    return EXIT_SUCCESS;
}
```

Листинг 3: Файл `src/matrix_utils.h` --- заголовочный файл, определяющий функции для работы с матрицами.
```c
#ifndef MATRIX_UTILS_H
#define MATRIX_UTILS_H

#include <stdio.h>
#include <stdlib.h>

// A square n-by-n matrix
typedef struct {
    int n;
    double **data;
} Matrix;

Matrix* allocate_matrix(int n);
void free_matrix(Matrix *matrix);
Matrix* read_matrix(const char *filename);
int write_matrix(const char *filename, const Matrix *matrix);
double compute_mean(const Matrix *matrix);
double compute_variance(const Matrix *matrix);
Matrix* transpose_matrix(const Matrix *matrix);
Matrix* multiply_matrices(const Matrix *A, const Matrix *B);
void add_scalar_to_diagonal(Matrix *matrix, double scalar);
void add_scalar_to_all_elements(Matrix *matrix, double scalar);

#endif // MATRIX_UTILS_H
```

Листинг 4: Файл `src/matrix_utils.c` --- реализации функций для работы с матрицами.
```c
#include "matrix_utils.h"
#include <math.h>
#include <omp.h>

Matrix* allocate_matrix(int n) {
    Matrix *matrix = (Matrix*)malloc(sizeof(Matrix));
    matrix->n = n;
    matrix->data = (double**)malloc(n * sizeof(double*));
    for (int i = 0; i < n; ++i) {
        matrix->data[i] = (double*)malloc(n * sizeof(double));
    }
    return matrix;
}

void free_matrix(Matrix *matrix) {
    for (int i = 0; i < matrix->n; ++i) {
        free(matrix->data[i]);
    }
    free(matrix->data);
    free(matrix);
}

Matrix* read_matrix(const char *filename) {
    FILE *fp = fopen(filename, "r");
    if (!fp) {
        perror("Failed to open matrix file");
        exit(EXIT_FAILURE);
    }
    int size;
    if (fscanf(fp, "%d", &size) == EOF) {
        fprintf(stderr, "Failed to read matrix size");
        exit(EXIT_FAILURE);
    }
    Matrix *matrix = allocate_matrix(size);
    for (int i = 0; i < size; ++i) {
        for (int j = 0; j < size; ++j) {
            if (fscanf(fp, "%lf", &matrix->data[i][j]) == EOF) {
                fprintf(
                    stderr, 
                    "Failed to read matrix element at pos (%d, %d)",
                    i, j
                );
                exit(EXIT_FAILURE);
            }
        }
    }
    fclose(fp);
    return matrix;
}

int write_matrix(const char *filename, const Matrix *matrix) {
    FILE *fp = fopen(filename, "w");
    if (!fp) {
        perror("Failed to write matrix file");
        return -1;
    }
    fprintf(fp, "%d\n", matrix->n);
    for (int i = 0; i < matrix->n; ++i) {
        for (int j = 0; j < matrix->n; ++j) {
            fprintf(fp, "%.6f ", matrix->data[i][j]);
        }
        fprintf(fp, "\n");
    }
    fclose(fp);
    return 0;
}

double compute_mean(const Matrix *matrix) {
    double sum = 0.0;
    int n = matrix->n;
    int total_elements = n * n;
    #pragma omp parallel for reduction(+:sum)
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < n; ++j) {
            sum += matrix->data[i][j];
        }
    }
    return sum / total_elements;
}

double compute_variance(const Matrix *matrix) {
    double mean = compute_mean(matrix);
    double variance = 0.0;
    int n = matrix->n;
    int total_elements = n * n;
    #pragma omp parallel for reduction(+:variance)
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < n; ++j) {
            double diff = matrix->data[i][j] - mean;
            variance += diff * diff;
        }
    }
    return variance / total_elements;
}

Matrix* transpose_matrix(const Matrix *matrix) {
    int n = matrix->n;
    Matrix *B_T = allocate_matrix(n);

    #pragma omp parallel for
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < n; ++j) {
            B_T->data[j][i] = matrix->data[i][j];
        }
    }

    return B_T;
}

Matrix* multiply_matrices(const Matrix *A, const Matrix *B) {
    int n = A->n;
    Matrix *result = allocate_matrix(n);

    // improve cache locality
    Matrix *B_T = transpose_matrix(B);

    #pragma omp parallel for
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < n; ++j) {
            double sum = 0.0;
            for (int k = 0; k < n; ++k) {
                sum += A->data[i][k] * B_T->data[j][k];
            }
            result->data[i][j] = sum;
        }
    }

    free_matrix(B_T);

    return result;
}

void add_scalar_to_diagonal(Matrix *matrix, double scalar) {
    int n = matrix->n;
    #pragma omp parallel for
    for (int i = 0; i < n; ++i) {
        matrix->data[i][i] += scalar;
    }
}

void add_scalar_to_all_elements(Matrix *matrix, double scalar) {
    int n = matrix->n;
    #pragma omp parallel for
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < n; ++j) {
            matrix->data[i][j] += scalar;
        }
    }
}
```

Листинг 5: Файл `Makefile` --- файл с правилами автоматической сборки проекта, используемый утилитой `make`.
```makefile
CC = gcc
CFLAGS = -O3 -fopenmp

all: main save_random_matrix

main: src/main.c src/matrix_utils.c
	$(CC) $(CFLAGS) -o main src/main.c src/matrix_utils.c

save_random_matrix: src/save_random_matrix.c src/matrix_utils.c
	$(CC) $(CFLAGS) -o save_random_matrix src/save_random_matrix.c \
	  src/matrix_utils.c

clean:
	rm -f main save_random_matrix
```


Листинг 6: Python-функция `run_experiment`, используемая для проведения замеров.
```python
def run_experiment(
        matrix_size,
        num_threads,
        seed=42,
        min_value=-1.0,
        max_value=1.0,
        reruns = 5,
        data_dir = 'data'
):
    """
    Returns the following triple:
      (mean_time, time_std, path_to_output_file)
    """
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    B_filename = os.path.join(data_dir, f'B_{matrix_size}.txt')
    C_filename = os.path.join(data_dir, f'C_{matrix_size}.txt')
    A_filename = os.path \
        .join(data_dir, f'A_{matrix_size}_{num_threads}.txt')
    
    subprocess.run(['./save_random_matrix', B_filename, \
                    str(matrix_size), str(min_value), \
                    str(max_value), str(seed)], check=True)
    
    subprocess.run(['./save_random_matrix', C_filename, \
                    str(matrix_size), str(min_value), \
                    str(max_value), str(seed+1)], check=True)
    
    times = []

    for _ in range(reruns):
        result = subprocess.run(['./main', B_filename, C_filename, \
                                A_filename, str(num_threads)], \
                                stdout=subprocess.PIPE, text=True)
        match = re.search(r'Computation time: ([\d\.]+) seconds', \
                          result.stdout)
        if match:
            times.append(float(match.group(1)))
        else:
            raise RuntimeError("Failed to extract computation time")
        
    return np.mean(times), np.std(times, ddof=1), A_filename
```

In [ ]:
# Save report, requires pandoc and no special characters in notebook outputs

!jupyter nbconvert --to markdown --no-input experiment.ipynb
!pandoc experiment.md -o report.pdf --pdf-engine=xelatex \
  -V title="Практическая работа \textnumero 1" \
  -V subtitle="Задание A-03 (OpenMP) \linebreak Методы и технологии высокопроизводительных вычислений I" \
  -V author="Муравьев И.В., группа 24.М71-мм" \
  -V date="\today" \
  -V mainfont="CMU Serif" \
  -V monofont="DejaVu Sans Mono" \
  -V colorlinks=true \
  -V linkcolor=blue \
  -V fontsize=14pt \
  -V documentclass=extarticle \
  -V geometry:margin=1in \
  -V lang=ru-ru \
  -fmarkdown-implicit_figures \
  --highlight-style=tango \
  -V header-includes="\
    \usepackage{titling}"
